In [1]:
import sys
import json
import folium
import pandas as pd
import numpy as np
import pickle as pkl
from shapely.ops import transform as transform_shapely

from pyproj import Transformer
from shapely.geometry import shape, Point
import geopandas as gpd
sys.path.append('..')

from src import STORAGE_PATH
from src.visualization import convert_dict_list_to_counts_dataframe, create_smooth_choropleth_layers

/home/macron/.pyenv/versions/3.9.9/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Load data

In [2]:
DATA_DIRECTORY = STORAGE_PATH / 'data'

accidents = gpd.read_file(DATA_DIRECTORY / 'accidents.geojson')
bikeroads = gpd.read_file(DATA_DIRECTORY / 'bikeroads.geojson')

In [3]:
with open(STORAGE_PATH / 'maps' / 'wroclaw-max.geojson', 'r') as f:
    wroclaw_map = json.load(f)

### Map accidents to regions

In [4]:
datasets = {'bike' : accidents[accidents['POJ_ROWER'] > 0], 'nonbike' : accidents[accidents['POJ_ROWER'] == 0]}

def find_region(geometry):
    for feature in wroclaw_map['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(geometry):
            return feature['properties']['osiedle']
    return ''

for acc_type in datasets:
    datasets[acc_type]['osiedle'] = datasets[acc_type]['geometry'].apply(lambda k : find_region(k))

/home/macron/.pyenv/versions/3.9.9/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/macron/.pyenv/versions/3.9.9/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


### Maps per accident type

In [5]:
folium_maps = []
grouped_sets = {}

in_proj = 'epsg:4326'
out_proj = 'epsg:2180'
transformer = Transformer.from_crs(in_proj, out_proj).transform

for acc_type in datasets:
    dataset = datasets[acc_type].groupby('osiedle', as_index=False).count()
    
    folium_map = folium.Map(location=[51.1, 17], zoom_start=12)
    
    def calc_countsn(row):
        for feature in wroclaw_map['features']:
            if feature['properties']['osiedle'] == row['osiedle']:
                return  row['POJ_SUMA'] / transform_shapely(transformer, shape(feature['geometry'])).area * 10000
    
    dataset['counts_n'] = dataset.apply(calc_countsn, axis=1)
                
    grouped_sets[acc_type] = dataset
    folium.Choropleth(
        geo_data=wroclaw_map,
        data=dataset,
        columns=['osiedle', 'counts_n'],
        name="choropleth",
        key_on='properties.osiedle',
        fill_color="Reds",
        bins=9,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f"Heatmap of {acc_type} accidents",
    ).add_to(folium_map)
    folium_maps.append(folium_map)

In [6]:
folium_maps[0]

In [7]:
folium_maps[1]

### Normalized difference

In [8]:
choropleth_df = grouped_sets['bike'].sort_values('osiedle')[['osiedle', 'counts_n']]
choropleth_df['nonbike_counts'] = grouped_sets['nonbike'].sort_values('osiedle')['POJ_SUMA']
avg_multiplier = choropleth_df['counts_n'].sum() / choropleth_df['nonbike_counts'].sum()
choropleth_df['predbike_counts'] = choropleth_df['nonbike_counts'] * avg_multiplier
choropleth_df['diffs'] = choropleth_df['counts_n'] / choropleth_df['predbike_counts']

In [9]:

folium_map = folium.Map(location=[51.1, 17], zoom_start=12)
bins = np.array(range(9)) * max(choropleth_df['diffs'].max(), -choropleth_df['diffs'].min()) / 8
folium.Choropleth(
            geo_data=wroclaw_map,
            data=choropleth_df,
            columns=['osiedle', 'diffs'],
            name="choropleth",
            key_on='properties.osiedle',
            fill_color="BuPu",
            bins=bins,
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f"Heatmap of {acc_type} accidents",
).add_to(folium_map)
folium_map

In [10]:
folium_map = folium.Map(location=[51.1, 17], zoom_start=12)
for layer in create_smooth_choropleth_layers(choropleth_df, 'osiedle', 'diffs', wroclaw_map, lambda k: k['properties']['osiedle'], '#4040FF', '#FF4040', mid_color='#FFFFFF', force_mid=1):
    layer.add_to(folium_map)
folium_map

### Non-regionalized heatmaps

In [18]:
from folium.plugins import HeatMap
folium_map = folium.Map(location=[51.1, 17], zoom_start=12)
points = []
for _, row in datasets['bike'].iterrows():
    points.append((row['geometry'].y, row['geometry'].x))

HeatMap(points, radius=6, blur=4).add_to(folium_map)
folium_map